In [ ]:
"""
okay, tasks for this notebook:

-make a teacher with descent class distribution
    --make a student of same architecture, train it (i have all these parts. ez.)
        ---using ints as input (default) and post-sigmoid as output
        ---remove last layer and use logits, then compare with sigmoide added on
        ---both?
    --swap the embedding layer for something that can take random noise.  add that to noiseKD? --on another nb
        ---this should enable us to train on noise, not just int indexes.  compare all
        
"""

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
import torch.optim as optim
from NoiseKD import Teacher, SimpleLanguageModel, slm_init_config, slm_model_config, count_parameters

In [2]:
SLM = SimpleLanguageModel(**slm_init_config)

In [3]:
count_parameters(SLM)

3105922

In [4]:
slm_init_config['vocab_size']

80

In [27]:
teacher_slm = Teacher(SLM,(160,))

In [ ]:
# for comparison, i need to be starting them off with the same init.  do that.

In [6]:
slm_model_config

{'dist_type': 'ints',
 'gen_m': 80,
 'gen_n': 2000,
 'gen_epochs': 50,
 'gen_lr': 0.003,
 'random_shuffle': 0.8,
 'out_type': 'one-hot'}

In [28]:
teacher_slm.configure(**slm_model_config)

Expected tensor for argument #1 'indices' to have one of the following scalar types: Long, Int; but got torch.FloatTensor instead (while checking arguments for embedding)
lets try ints!


Configuring Teacher:: 100%|█████████████████████| 50/50 [01:42<00:00,  2.05s/it]

Teacher Configured


In [ ]:
args = { 'val_train' : "train"
                      , 'n' : 100_000
                      , 'dist_type' : 'ints'
                      , 'm' : slm_init_config['vocab_size']
                      , 'std': 1.0
        }
teacher_slm.generate_data(**args)

In [29]:
args = { 'val_train' : "val"
                      , 'n' : 10_000
                      , 'dist_type' : 'ints'
                      , 'm' : slm_init_config['vocab_size']
                      , 'std': 1.0
        }
teacher_slm.generate_data(**args)

Generating val data :: 100%|██████████████████| 200/200 [00:05<00:00, 38.35it/s]


In [30]:
student = SimpleLanguageModel(**slm_init_config)

In [31]:
student

SimpleLanguageModel(
  (embedding): Embedding(80, 16)
  (transformer_encoder): TransformerEncoder(
    (transformer): TransformerEncoder(
      (layers): ModuleList(
        (0): TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=16, out_features=16, bias=True)
          )
          (linear1): Linear(in_features=16, out_features=11, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
          (linear2): Linear(in_features=11, out_features=16, bias=True)
          (norm1): LayerNorm((16,), eps=1e-05, elementwise_affine=True)
          (norm2): LayerNorm((16,), eps=1e-05, elementwise_affine=True)
          (dropout1): Dropout(p=0.1, inplace=False)
          (dropout2): Dropout(p=0.1, inplace=False)
        )
        (1): TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=16, out_features=16, bias=True)
       

In [ ]:
##to train form the teacher object was written already

In [11]:
num_epochs = 2
batch_size = 32
learning_rate = 0.009 #0.00025 this shit is slooooow
momentum = 0.95
criterion = nn.MSELoss()
optimizer = optim.Adam(student.parameters(), lr=learning_rate)

optimizer = optim.SGD(student.parameters(), lr=learning_rate, momentum=momentum)
data = list(zip(teacher_slm.train_inputs, teacher_slm.train_targets))
input_tensors = torch.stack([torch.Tensor(x[0]) for x in data])
target_tensors = torch.stack([torch.Tensor(x[1]) for x in data])
dataset = TensorDataset(input_tensors, target_tensors)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

val_data = list(zip(teacher_slm.val_inputs, teacher_slm.val_targets))
val_input_tensors = torch.stack([torch.Tensor(x[0]) for x in val_data])
val_target_tensors = torch.stack([torch.Tensor(x[1]) for x in val_data])
val_dataset = TensorDataset(val_input_tensors, val_target_tensors)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

RuntimeError: stack expects a non-empty TensorList

In [ ]:
#conventional train, repeats throught the dataset
accuracy_threshold = 0.5
print_every = 1 #its working, its just lots of data mama.
validation_every = 1
losses = []  # List to store losses
accuracies = []

if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

student = student.to(device)
for epoch in range(num_epochs):
    student.train()
    total_loss = 0
    correct_predictions = 0
    total_samples = 0
    
    for input_batch, target_batch in dataloader:
        optimizer.zero_grad()  # Zero the gradients
        input_batch = input_batch.to(device)
        target_batch = target_batch.to(device)
        output = student(input_batch)  # Forward pass
        loss = criterion(output, target_batch)  # Compute the loss
        loss.backward()  # Backpropagation
        optimizer.step()  # Update the weights
        total_loss += loss.item()
        #add early stopping...
        #and validation at each step.  
        # Calculate accuracy
        predictions = (output > accuracy_threshold).float()  # Assuming a threshold of 0.5 for binary classification
        correct_predictions += (predictions == target_batch).sum().item()
        total_samples += input_batch.size(0)

    # Print the average loss for this epoch
    avg_loss = total_loss / len(dataloader)
    losses.append(avg_loss)
        
    accuracy = correct_predictions / total_samples
    accuracies.append(accuracy)

    if (epoch + 1) % print_every == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.8f}, Train Accuracy: {accuracy:.4f}')

    if (epoch + 1) % validation_every == 0:
        student.eval()
        
        total_val_samples = 0
        correct_val_predictions = 0

        with torch.no_grad():
            for val_input_batch, val_target_batch in val_dataloader:
                val_input_batch = val_input_batch.to(device)
                val_target_batch = val_target_batch.to(device)
                val_output = student(val_input_batch)
                val_predictions = (val_output > accuracy_threshold).float()  # Assuming a threshold of 0.5 for binary classification
                correct_val_predictions += (val_predictions == val_target_batch).sum().item()
                total_val_samples += val_input_batch.size(0)

        # Calculate validation accuracy
        val_accuracy = correct_val_predictions / total_val_samples

        # Print the validation accuracy for this epoch
        print(f'\t\tValidation Accuracy: {val_accuracy:.4f}')

        # Set the model back to training mode
        student.train()
            
            

In [33]:
#teacher_slm.model #make a .init() function in NoiseKD
for layer in student.modules():
    if isinstance(layer, nn.Linear):
        nn.init.xavier_uniform_(layer.weight)  # You can choose different initialization methods
        nn.init.constant_(layer.bias, 0.0)

In [34]:

learning_rate = 0.001 #0.00025 this shit is slooooow
momentum = 0.95
criterion = nn.MSELoss()
optimizer = optim.Adam(student.parameters(), lr=learning_rate)

#optimizer = optim.SGD(student.parameters(), lr=learning_rate, momentum=momentum)
#no train data loader here.
val_data = list(zip(teacher_slm.val_inputs, teacher_slm.val_targets))
val_input_tensors = torch.stack([torch.Tensor(x[0]) for x in val_data])
val_target_tensors = torch.stack([torch.Tensor(x[1]) for x in val_data])
val_dataset = TensorDataset(val_input_tensors, val_target_tensors)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
accuracy_threshold = 0.5
print_every = 1 #its working, its just lots of data mama.
validation_every = 1

batches_per_epoch = 10
data_per_batch = 500
num_epochs = 400
batch_size = 100
gen_args  = { 'val_train' : "train"
              , 'n' : data_per_batch
              , 'dist_type' : 'ints'
              , 'm' : slm_init_config['vocab_size']
              , 'std': 1.0
              , 'display_progress' : False
        }

losses = []  # List to store losses
accuracies = []

if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

student = student.to(device)
for epoch in range(num_epochs):
    student.train()
    total_loss = 0
    correct_predictions = 0
    total_samples = 0
    
    for i in range(batches_per_epoch):
        optimizer.zero_grad()  # Zero the gradients
        
        teacher_slm.generate_data(**gen_args)
        data_e = list(zip(teacher_slm.train_inputs, teacher_slm.train_targets))
        input_tensors_e = torch.stack([torch.Tensor(x[0]) for x in data_e])
        target_tensors_e = torch.stack([torch.Tensor(x[1]) for x in data_e])
        dataset_e = TensorDataset(input_tensors_e, target_tensors_e)
        dataloader_e = DataLoader(dataset_e, batch_size=batch_size, shuffle=True)
        
        for input_batch_e, target_batch_e in dataloader_e:
            optimizer.zero_grad()  # Zero the gradients
            input_batch_e = input_batch_e.to(device)
            target_batch_e = target_batch_e.to(device)
            output = student(input_batch_e)  # Forward pass
            loss = criterion(output, target_batch_e)  # Compute the loss
            loss.backward()  # Backpropagation
            optimizer.step()  # Update the weights
            total_loss += loss.item()
            #add early stopping...
            #and validation at each step.  
            # Calculate accuracy
            predictions = (output > accuracy_threshold).float()  # Assuming a threshold of 0.5 for binary classification
            correct_predictions += (predictions == target_batch_e).sum().item()
            total_samples += input_batch_e.size(0)

    # Print the average loss for this epoch
    avg_loss = total_loss / ( batches_per_epoch * data_per_batch)
    losses.append(avg_loss)
        
    accuracy = correct_predictions / total_samples
    accuracies.append(accuracy)

    if (epoch + 1) % print_every == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.8f}, Train Accuracy: {accuracy:.4f}')

    if (epoch + 1) % validation_every == 0:
        student.eval()
        
        total_val_samples = 0
        correct_val_predictions = 0

        with torch.no_grad():
            for val_input_batch, val_target_batch in val_dataloader:
                val_input_batch = val_input_batch.to(device)
                val_target_batch = val_target_batch.to(device)
                val_output = student(val_input_batch)
                val_predictions = (val_output > accuracy_threshold).float()  # Assuming a threshold of 0.5 for binary classification
                correct_val_predictions += (val_predictions == val_target_batch).sum().item()
                total_val_samples += val_input_batch.size(0)

        # Calculate validation accuracy
        val_accuracy = correct_val_predictions / total_val_samples

        # Print the validation accuracy for this epoch
        print(f'\t\tValidation Accuracy: {val_accuracy:.4f}')

        # Set the model back to training mode
        student.train()
            
            

Epoch [1/400], Loss: 0.00012196, Train Accuracy: 78.8936
		Validation Accuracy: 78.9142
Epoch [2/400], Loss: 0.00011925, Train Accuracy: 78.9142
		Validation Accuracy: 78.9189
Epoch [3/400], Loss: 0.00011563, Train Accuracy: 78.9118
		Validation Accuracy: 78.8671
Epoch [4/400], Loss: 0.00011203, Train Accuracy: 78.9206
		Validation Accuracy: 78.9190
Epoch [5/400], Loss: 0.00011121, Train Accuracy: 78.9130
		Validation Accuracy: 78.9189
Epoch [6/400], Loss: 0.00011064, Train Accuracy: 78.9166
		Validation Accuracy: 78.9189
Epoch [7/400], Loss: 0.00011074, Train Accuracy: 78.9118
		Validation Accuracy: 78.9189
Epoch [8/400], Loss: 0.00011043, Train Accuracy: 78.9164
		Validation Accuracy: 78.9191
Epoch [9/400], Loss: 0.00011069, Train Accuracy: 78.9154
		Validation Accuracy: 78.9185


KeyboardInterrupt: 

In [ ]:
#all unique train, 

##expand this then try collab
#scc? or na? that competes with the other stuff i bet

print_every = 1
batch_eternal = 50
data_n = 5000
num_epochs = 400
losses = []  # List to store losses

for epoch in range(num_epochs):
    total_loss = 0
    teacher.generate_data(
    "train"
    ,data_n
    ,'normal'
    , m =0.0
    , std=1.0
    )
    data = list(zip(teacher.train_inputs, teacher.train_targets))
    input_tensors = torch.stack([torch.Tensor(x[0]) for x in data])
    target_tensors = torch.stack([torch.Tensor(x[1]) for x in data])
    dataset = TensorDataset(input_tensors, target_tensors)
    dataloader = DataLoader(dataset, batch_size=batch_eternal, shuffle=True)
    
    for input_batch, target_batch in dataloader:
        optimizer.zero_grad()  # Zero the gradients
        output = student(input_batch)  # Forward pass
        loss = criterion(output, target_batch)  # Compute the loss
        loss.backward()  # Backpropagation
        optimizer.step()  # Update the weights
        total_loss += loss.item()

    # Print the average loss for this epoch
    avg_loss = total_loss / len(dataloader)
    losses.append(avg_loss)
    if (epoch + 1) % print_every == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.8f}')